In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1eZ5s4Ln6YU2bla_phSzWPNZl4m1zUzen", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/05_00_intro.mp3"))


# Capstone: Build a Personalized FAQ Chatbot with Context Engineering

*Part 5 (Capstone) of the Vizuara series on Context Engineering for LLMs*
*Estimated time: 45 minutes*

You have spent four notebooks mastering every dimension of context engineering — token budgets, failure modes, retrieval-augmented generation, and the four context strategies. Now it is time to combine all of them into a single, working system that feels genuinely useful.

In this capstone, you will build a personalized FAQ chatbot for Vizuara AI Labs. It scrapes a website, chunks and embeds the content, retrieves relevant documents for every question, remembers who the user is across turns, calls tools when needed, and assembles optimal context using every technique from Notebooks 1-4. By the end, you will watch it hold a multi-turn conversation that feels surprisingly intelligent — and you will understand exactly why, because you built every piece yourself.

The best part: you can swap vizuara.ai for **any** website URL and instantly have a custom chatbot for that site.

In [ ]:
#@title 🎧 Listen: Ai Assistant
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_02_ai_assistant.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/context-engineering-for-llms/practice/5/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Code Walkthrough: Install Deps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_03_install_deps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Setup — install all dependencies
!pip install -q google-generativeai sentence-transformers beautifulsoup4 requests

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Any, Callable
import textwrap
import re
import json
import time
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11

print("Setup complete! Ready to build the capstone chatbot.")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_04_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

Every AI chatbot you interact with — customer support bots, coding assistants, documentation helpers — faces the same core challenge: **what do I put in the context window?**

A naive chatbot just concatenates the user's question with a system prompt and hopes for the best. A well-engineered chatbot:

| Dimension | Naive Approach | Context-Engineered Approach |
|-----------|---------------|---------------------------|
| **Knowledge** | Stuff everything into the prompt | RAG: retrieve only relevant chunks |
| **Memory** | No memory — every turn is independent | Track user profile, past questions, key facts |
| **Tools** | Hardcode everything | Dynamic tool dispatch based on intent |
| **Token Budget** | Hope it fits | Explicit budget allocation + compression |
| **Failure Modes** | Ignore them | Detect and recover from stale, conflicting, or missing context |

Today we build the right column. All of it.

In [ ]:
#@title 🎧 What to Look For: Architecture Viz Before
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_05_architecture_viz_before.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let's visualize what we're building: the full architecture

fig, ax = plt.subplots(1, 1, figsize=(14, 9))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title("Capstone Architecture: Personalized FAQ Chatbot",
             fontsize=16, fontweight='bold', pad=20)

# User input at the top
user_box = plt.Rectangle((5.5, 8.5), 3, 1.0, linewidth=2,
                           edgecolor='#2196F3', facecolor='#E3F2FD',
                           borderRadius=0.1)
ax.add_patch(user_box)
ax.text(7, 9.0, "User Message", ha='center', va='center',
        fontsize=12, fontweight='bold', color='#1565C0')

# Arrow down from user
ax.annotate('', xy=(7, 8.5), xytext=(7, 8.0),
           arrowprops=dict(arrowstyle='->', color='#333', lw=2))

# Context Assembler in the middle
assembler_box = plt.Rectangle((3.5, 5.8), 7, 2.0, linewidth=2.5,
                                edgecolor='#FF9800', facecolor='#FFF8E1',
                                linestyle='-')
ax.add_patch(assembler_box)
ax.text(7, 7.2, "Context Assembler", ha='center', va='center',
        fontsize=13, fontweight='bold', color='#E65100')
ax.text(7, 6.5, "Token Budget | Health Check | Write/Select/Compress/Isolate",
        ha='center', va='center', fontsize=9, color='#BF360C',
        style='italic')

# Four input sources
sources = [
    (0.5, 3.5, "Knowledge\nBase\n(RAG)", '#4CAF50', '#E8F5E9'),
    (4.0, 3.5, "Conversation\nMemory", '#9C27B0', '#F3E5F5'),
    (7.5, 3.5, "Tool\nResults", '#F44336', '#FFEBEE'),
    (11.0, 3.5, "System\nPrompt", '#607D8B', '#ECEFF1'),
]

for x, y, label, edge_color, face_color in sources:
    box = plt.Rectangle((x, y), 2.5, 1.8, linewidth=2,
                          edgecolor=edge_color, facecolor=face_color)
    ax.add_patch(box)
    ax.text(x + 1.25, y + 0.9, label, ha='center', va='center',
            fontsize=10, fontweight='bold', color=edge_color)
    # Arrow up to assembler
    ax.annotate('', xy=(x + 1.25, 5.8), xytext=(x + 1.25, 5.3),
               arrowprops=dict(arrowstyle='->', color=edge_color, lw=1.5))

# Arrow down to LLM
ax.annotate('', xy=(7, 5.8), xytext=(7, 3.0),
           arrowprops=dict(arrowstyle='->', color='#333', lw=2))

# LLM box at the bottom
llm_box = plt.Rectangle((4.5, 1.2), 5, 1.5, linewidth=2.5,
                          edgecolor='#1A237E', facecolor='#E8EAF6')
ax.add_patch(llm_box)
ax.text(7, 1.95, "Gemini 2.0 Flash", ha='center', va='center',
        fontsize=12, fontweight='bold', color='#1A237E')
ax.text(7, 1.5, "Assembled Context  →  Response",
        ha='center', va='center', fontsize=9, color='#283593')

# Arrow from LLM to response
ax.annotate('', xy=(7, 1.2), xytext=(7, 0.7),
           arrowprops=dict(arrowstyle='->', color='#333', lw=2))
ax.text(7, 0.4, "Personalized, Context-Aware Response",
        ha='center', va='center', fontsize=11, fontweight='bold',
        color='#333')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Architecture Viz After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_06_architecture_viz_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_07_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Think of a great hotel concierge. When a guest walks up and says "What's good for dinner?", the concierge doesn't give a generic answer. They remember:

- **Who you are**: "Ms. Chen, welcome back!" (memory)
- **What you like**: "Last time you loved the Italian place on 5th" (profile)
- **What's available**: They check their binder of restaurant info (knowledge retrieval)
- **What tools they have**: They can call restaurants to check availability (tool use)
- **What's relevant**: They don't read you their entire binder — just the Italian and seafood sections (context selection)

Our chatbot works the same way. The "magic" of a great chatbot is not a better model — it is better context engineering.

### What We Will Build — Component by Component

1. **Knowledge Base** — Scrape a website + load curated content, chunk it into retrievable pieces
2. **Vector Store** — Embed chunks and find the most relevant ones for each question
3. **Conversation Memory** — Track user profile, conversation history, and extracted facts
4. **Tool Registry** — Register callable tools (course lookup, FAQ search, contact info)
5. **Context Assembler** — Allocate token budget, run health checks, apply the four strategies
6. **Chatbot** — Tie everything together into a conversational loop

Let's build each one.

In [ ]:
#@title 🎧 Listen: Setting Up Llm
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_08_setting_up_llm.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Architecture — Setting Up the LLM

We use Google Gemini (free tier). You will need an API key from [https://aistudio.google.com](https://aistudio.google.com).

In Google Colab, store your key using the Secrets panel (key icon in the left sidebar) with the name `GOOGLE_API_KEY`.

In [ ]:
#@title 🎧 Code Walkthrough: Configure Gemini
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_09_configure_gemini.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini with your API key
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

# We'll use gemini-2.0-flash — fast, capable, and free-tier friendly
MODEL_NAME = 'gemini-2.0-flash'

# Quick test to make sure the API is working
model = genai.GenerativeModel(MODEL_NAME)
response = model.generate_content("Say 'Context engineering is ready!' in exactly those words.")
print(f"Model test: {response.text.strip()}")
print(f"Using model: {MODEL_NAME}")

In [ ]:
#@title 🎧 Listen: Embedding Model Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_10_embedding_model_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


We also need a sentence embedding model for our vector store. We use `all-MiniLM-L6-v2` — it is small, fast, and runs on CPU without issues.

In [ ]:
#@title 🎧 Code Walkthrough: Load Embedding Model
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_11_load_embedding_model.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
from sentence_transformers import SentenceTransformer

# Load embedding model (runs on CPU, ~80MB download)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Quick test
test_embedding = embedding_model.encode(["Hello world"])
print(f"Embedding model loaded!")
print(f"Embedding dimension: {test_embedding.shape[1]}")
print(f"Test embedding norm: {np.linalg.norm(test_embedding[0]):.4f}")

In [ ]:
#@title 🎧 Transition: Component1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_12_component1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

### 4.1 Component 1: Knowledge Base

The knowledge base has two sources: a curated dictionary of Vizuara information (provided below), and optionally scraped web content. We chunk all content into smaller pieces suitable for embedding and retrieval.

In [ ]:
#@title 🎧 Code Walkthrough: Curated Kb
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_13_curated_kb.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Curated Vizuara knowledge base — this is what makes the chatbot knowledgeable

VIZUARA_KNOWLEDGE_BASE = [
    {
        "topic": "About Vizuara",
        "content": "Vizuara AI Labs is an educational platform that teaches AI and machine learning concepts through in-depth articles, hands-on Google Colab notebooks, and real-world case studies. Founded by Raj Dandekar, Vizuara's mission is to make cutting-edge AI research accessible to everyone — from students to practitioners. The name 'Vizuara' combines 'vision' and 'aura', reflecting the goal of illuminating complex AI concepts with clarity."
    },
    {
        "topic": "Courses and Content",
        "content": "Vizuara offers courses on topics including: Reinforcement Learning and DeepSeek-R1, Diffusion Models for Video Generation, World Action Models for Robotics, Vision-Language-Action Models, 5D Parallelism for GPU Programming, Agentic RAG, Context Engineering for LLMs, and more. Each course includes a detailed Substack article, multiple Google Colab teaching notebooks, and case studies with real datasets."
    },
    {
        "topic": "Teaching Philosophy",
        "content": "Vizuara's teaching philosophy is 'from first principles'. Every concept is built from the ground up — students implement core algorithms from scratch using PyTorch, understand the mathematics behind each technique, and produce tangible outputs (trained models, visualizations, working systems). No black boxes. No copy-pasting from StackOverflow. The goal is deep understanding, not surface-level familiarity."
    },
    {
        "topic": "Substack Publication",
        "content": "Vizuara publishes in-depth technical articles on Substack at vizuara.substack.com. Each article is 3,000-8,000 words long, includes custom figures, LaTeX equations, and runnable code. Articles cover the latest AI research papers and translate them into practical, implementable knowledge. The writing style is warm, pedagogical, and assumes an intelligent reader who wants to understand WHY, not just HOW."
    },
    {
        "topic": "Google Colab Notebooks",
        "content": "Each Vizuara article comes with companion Google Colab notebooks that let students build the concepts hands-on. Notebooks follow a structured pedagogy: motivation → intuition → math → incremental implementation → exercises → final output. All notebooks run on Google Colab's free T4 GPU and typically take 30-45 minutes to complete. Students implement everything from scratch — no high-level wrappers that hide the core mechanics."
    },
    {
        "topic": "Case Studies",
        "content": "Vizuara case studies connect AI theory to real-world business problems. Each case study uses real, publicly available datasets (never toy data), frames the problem with business context, and guides students through building a complete ML solution. Case studies include a Colab notebook, production design considerations, and are solvable on a T4 GPU in under 90 minutes."
    },
    {
        "topic": "Pricing and Access",
        "content": "Vizuara's Substack articles are free to read. Google Colab notebooks are freely accessible. The platform is committed to making AI education accessible to everyone regardless of financial situation. Students can access all content at vizuara.substack.com and the companion notebooks through Google Colab links provided in each article."
    },
    {
        "topic": "Technology Stack",
        "content": "The Vizuara course platform is built with Next.js 16, React 19, Tailwind CSS v4, and Framer Motion. The AI generation pipeline uses Claude (Anthropic) for content generation and Google Gemini for figure creation. Notebooks primarily use PyTorch as the deep learning framework, with matplotlib for visualizations."
    },
    {
        "topic": "Contact and Community",
        "content": "Students can reach Vizuara through the Substack publication comments, or through the website at vizuara.ai. Vizuara encourages questions and discussion — the goal is to build a community of learners who help each other understand complex AI concepts."
    },
    {
        "topic": "FAQ - Prerequisites",
        "content": "Most Vizuara courses assume familiarity with Python programming and basic linear algebra. Some courses require knowledge of calculus and probability. Each article and notebook clearly states its prerequisites. If you can write a Python function and understand matrix multiplication, you can follow most Vizuara content."
    },
    {
        "topic": "FAQ - GPU Requirements",
        "content": "All Vizuara notebooks are designed to run on Google Colab's free T4 GPU. You do not need your own GPU or any paid compute. Simply open the Colab link, select 'GPU' under Runtime → Change runtime type, and run the cells. Training times are kept under 10 minutes per notebook."
    },
    {
        "topic": "FAQ - How to Get Started",
        "content": "Start with any topic that interests you — each course is self-contained. Read the Substack article first to understand the big picture, then work through the companion notebooks. The notebooks build concepts incrementally, so start from Notebook 01 and work through in order. Don't skip the intuition-building sections — they make the math and code much easier to understand."
    }
]

print(f"Loaded {len(VIZUARA_KNOWLEDGE_BASE)} curated knowledge entries")
for entry in VIZUARA_KNOWLEDGE_BASE:
    print(f"  - {entry['topic']}: {len(entry['content'])} chars")

In [ ]:
#@title 🎧 Listen: Knowledgebase Class Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_14_knowledgebase_class_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now we build the `KnowledgeBase` class that handles loading curated content, scraping web pages, and chunking everything into retrieval-ready pieces.

In [ ]:
#@title 🎧 Code Walkthrough: Knowledgebase Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_15_knowledgebase_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import requests
from bs4 import BeautifulSoup

class KnowledgeBase:
    """
    Manages the chatbot's knowledge: curated entries + scraped web content.
    Chunks all content into pieces suitable for embedding and retrieval.
    """

    def __init__(self):
        self.documents = []  # List of {"text": ..., "source": ..., "topic": ...}
        self.chunks = []     # List of {"text": ..., "source": ..., "topic": ..., "chunk_id": ...}

    def load_curated(self, knowledge_list: List[Dict]) -> int:
        """Load curated knowledge base entries."""
        for entry in knowledge_list:
            self.documents.append({
                "text": entry["content"],
                "source": "curated",
                "topic": entry["topic"]
            })
        print(f"Loaded {len(knowledge_list)} curated documents")
        return len(knowledge_list)

    def scrape_website(self, url: str, max_pages: int = 5) -> int:
        """
        Scrape a URL and add content to the knowledge base.
        Extracts text from paragraphs, headings, and list items.
        """
        scraped_count = 0
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (compatible; VizuaraBot/1.0; Educational)'
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Remove script and style elements
            for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
                tag.decompose()

            # Extract text from meaningful elements
            elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'li', 'td',
                                       'article', 'section', 'div'])

            page_text = []
            for elem in elements:
                text = elem.get_text(strip=True)
                if len(text) > 30:  # Skip very short fragments
                    page_text.append(text)

            if page_text:
                # Combine into a single document
                full_text = "\n".join(page_text)
                self.documents.append({
                    "text": full_text,
                    "source": f"scraped:{url}",
                    "topic": f"Web content from {url}"
                })
                scraped_count = 1
                print(f"Scraped {url}: {len(full_text)} chars, "
                      f"{len(page_text)} text blocks")
            else:
                print(f"No substantial text found at {url}")

        except requests.RequestException as e:
            print(f"Could not scrape {url}: {e}")
            print("(This is fine — the curated knowledge base is sufficient)")

        return scraped_count

    def chunk(self, chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
        """
        Chunk all documents into smaller pieces for embedding.
        Uses character-level chunking with overlap for context continuity.
        """
        self.chunks = []
        chunk_id = 0

        for doc in self.documents:
            text = doc["text"]
            # Split into sentences first for cleaner chunk boundaries
            sentences = re.split(r'(?<=[.!?])\s+', text)

            current_chunk = ""
            for sentence in sentences:
                if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
                    self.chunks.append({
                        "text": current_chunk.strip(),
                        "source": doc["source"],
                        "topic": doc["topic"],
                        "chunk_id": chunk_id
                    })
                    chunk_id += 1
                    # Keep overlap: take the last `overlap` chars of the current chunk
                    if len(current_chunk) > overlap:
                        current_chunk = current_chunk[-overlap:] + " " + sentence
                    else:
                        current_chunk = sentence
                else:
                    current_chunk = (current_chunk + " " + sentence).strip()

            # Don't forget the last chunk
            if current_chunk.strip():
                self.chunks.append({
                    "text": current_chunk.strip(),
                    "source": doc["source"],
                    "topic": doc["topic"],
                    "chunk_id": chunk_id
                })
                chunk_id += 1

        print(f"Created {len(self.chunks)} chunks from {len(self.documents)} documents")
        print(f"Average chunk size: {np.mean([len(c['text']) for c in self.chunks]):.0f} chars")
        return self.chunks

    def get_stats(self) -> Dict:
        """Return statistics about the knowledge base."""
        return {
            "num_documents": len(self.documents),
            "num_chunks": len(self.chunks),
            "total_chars": sum(len(c["text"]) for c in self.chunks),
            "avg_chunk_size": np.mean([len(c["text"]) for c in self.chunks]) if self.chunks else 0,
            "sources": list(set(c["source"] for c in self.chunks))
        }

In [ ]:
#@title 🎧 Code Walkthrough: Build Kb
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_16_build_kb.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Build our knowledge base
kb = KnowledgeBase()

# Load curated content (always available)
kb.load_curated(VIZUARA_KNOWLEDGE_BASE)

# Try to scrape the live website (may fail — that's fine)
kb.scrape_website("https://vizuara.ai")

# Chunk everything
chunks = kb.chunk(chunk_size=400, overlap=50)

# Show stats
stats = kb.get_stats()
print(f"\nKnowledge Base Stats:")
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
#@title 🎧 Listen: Peek Chunks Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_17_peek_chunks_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let's peek at a few chunks to make sure the chunking looks reasonable.

In [ ]:
#@title 🎧 Code Walkthrough: Peek Chunks Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_18_peek_chunks_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Inspect the first 5 chunks
print("Sample chunks:")
print("=" * 70)
for i, chunk in enumerate(chunks[:5]):
    print(f"\nChunk {i} (source: {chunk['source']}, topic: {chunk['topic']})")
    print(f"  Length: {len(chunk['text'])} chars")
    print(f"  Text: {chunk['text'][:120]}...")
    print("-" * 70)

In [ ]:
#@title 🎧 Transition: Component2 Vectorstore Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_19_component2_vectorstore_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Component 2: Vector Store

The vector store embeds all chunks and provides similarity search. This is the RAG engine — given a user query, it finds the most relevant pieces of knowledge.

In [ ]:
#@title 🎧 Code Walkthrough: Vectorstore Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_20_vectorstore_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class VectorStore:
    """
    Simple but functional vector store using sentence-transformers.
    Embeds text chunks and provides cosine similarity search.
    """

    def __init__(self, model: SentenceTransformer):
        self.model = model
        self.embeddings = None        # numpy array of shape (N, dim)
        self.texts = []               # raw text for each entry
        self.metadata = []            # metadata dicts for each entry
        self.embedding_dim = model.get_sentence_embedding_dimension()

    def add(self, texts: List[str], metadata: Optional[List[Dict]] = None):
        """Embed and store texts with optional metadata."""
        if not texts:
            return

        # Compute embeddings in batch
        new_embeddings = self.model.encode(texts, show_progress_bar=False,
                                            normalize_embeddings=True)

        if self.embeddings is None:
            self.embeddings = new_embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, new_embeddings])

        self.texts.extend(texts)

        if metadata:
            self.metadata.extend(metadata)
        else:
            self.metadata.extend([{} for _ in texts])

        print(f"Added {len(texts)} entries. Total: {len(self.texts)}")

    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        """
        Return top-K most similar texts using cosine similarity.
        Returns list of {"text": ..., "score": ..., "metadata": ...}
        """
        if self.embeddings is None or len(self.texts) == 0:
            return []

        # Embed the query
        query_embedding = self.model.encode([query], normalize_embeddings=True)

        # Cosine similarity (embeddings are already normalized)
        similarities = np.dot(self.embeddings, query_embedding.T).flatten()

        # Get top-K indices
        top_indices = np.argsort(similarities)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append({
                "text": self.texts[idx],
                "score": float(similarities[idx]),
                "metadata": self.metadata[idx]
            })

        return results

    def get_stats(self) -> Dict:
        """Return vector store statistics."""
        return {
            "num_entries": len(self.texts),
            "embedding_dim": self.embedding_dim,
            "memory_mb": (self.embeddings.nbytes / 1e6) if self.embeddings is not None else 0
        }

In [ ]:
#@title 🎧 Code Walkthrough: Build Vectorstore
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_21_build_vectorstore.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Build the vector store from our chunked knowledge base
vector_store = VectorStore(embedding_model)

# Add all chunks
chunk_texts = [c["text"] for c in chunks]
chunk_metadata = [{"source": c["source"], "topic": c["topic"],
                    "chunk_id": c["chunk_id"]} for c in chunks]

vector_store.add(chunk_texts, chunk_metadata)

# Show stats
vs_stats = vector_store.get_stats()
print(f"\nVector Store Stats:")
for key, value in vs_stats.items():
    print(f"  {key}: {value}")

In [ ]:
#@title 🎧 Listen: Test Retrieval Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_22_test_retrieval_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let's test the retrieval with a few sample queries to make sure it works.

In [ ]:
#@title 🎧 Code Walkthrough: Test Retrieval Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_23_test_retrieval_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test retrieval quality
test_queries = [
    "What courses does Vizuara offer?",
    "Do I need a GPU?",
    "How do I get started?",
    "Who founded Vizuara?",
    "Are the notebooks free?"
]

print("Retrieval Quality Test")
print("=" * 70)

for query in test_queries:
    results = vector_store.search(query, top_k=2)
    print(f"\nQuery: \"{query}\"")
    for i, r in enumerate(results):
        print(f"  Result {i+1} (score={r['score']:.3f}, topic={r['metadata']['topic']}):")
        print(f"    {r['text'][:100]}...")
    print("-" * 70)

In [ ]:
#@title 🎧 What to Look For: Viz1 Retrieval Heatmap Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_24_viz1_retrieval_heatmap_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Visualization Checkpoint 1: Retrieval Similarity Heatmap

Let's visualize how well our vector store distinguishes between different types of questions. We will embed several queries and compute their similarity to all chunks, creating a heatmap.

In [ ]:
#@title 🎧 What to Look For: Viz1 Retrieval Heatmap Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_25_viz1_retrieval_heatmap_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualization: Query-Chunk similarity heatmap
viz_queries = [
    "What courses are available?",
    "Do I need a GPU to run notebooks?",
    "Who is the founder?",
    "What is the teaching philosophy?",
    "How much does it cost?",
    "What programming language is used?"
]

# Embed queries
query_embeddings = embedding_model.encode(viz_queries, normalize_embeddings=True)

# Compute similarities against all chunks
similarity_matrix = np.dot(query_embeddings, vector_store.embeddings.T)

# Get chunk labels (truncated topic names)
chunk_labels = [c["topic"][:20] for c in chunks]

fig, ax = plt.subplots(1, 1, figsize=(14, 5))
im = ax.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto',
               vmin=0, vmax=1)

ax.set_yticks(range(len(viz_queries)))
ax.set_yticklabels([q[:40] for q in viz_queries], fontsize=9)
ax.set_xlabel("Knowledge Base Chunks", fontsize=11)
ax.set_ylabel("User Queries", fontsize=11)
ax.set_title("RAG Retrieval: Query-Chunk Similarity Heatmap",
             fontsize=14, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label("Cosine Similarity", fontsize=10)

# Mark the top-1 chunk for each query
for i in range(len(viz_queries)):
    best_j = np.argmax(similarity_matrix[i])
    ax.plot(best_j, i, 'w*', markersize=12, markeredgecolor='black',
            markeredgewidth=1)

ax.text(0.02, -0.18, "Stars mark the top-1 retrieval for each query",
        transform=ax.transAxes, fontsize=9, style='italic', color='#555')

plt.tight_layout()
plt.show()

print("\n✅ The heatmap shows each query lighting up different chunks —")
print("   our retrieval is correctly matching questions to relevant content.")

In [ ]:
#@title 🎧 Transition: Component3 Memory Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_26_component3_memory_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Component 3: Conversation Memory

This is what makes the chatbot feel personal. The memory system tracks:
- **User profile**: name, interests, experience level, goals
- **Conversation history**: full turn-by-turn record
- **Key facts**: important things extracted from the conversation

In [ ]:
#@title 🎧 Code Walkthrough: Memory Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_27_memory_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class ConversationMemory:
    """
    Persistent memory across conversation turns.
    Tracks user profile, conversation history, and extracted key facts.
    """

    def __init__(self):
        self.user_profile = {}          # name, interests, experience, goals
        self.conversation_history = []  # list of {"role": ..., "content": ...}
        self.key_facts = []             # important facts from the conversation
        self.turn_count = 0

    def add_turn(self, role: str, content: str):
        """Add a conversation turn (role='user' or 'assistant')."""
        self.conversation_history.append({
            "role": role,
            "content": content,
            "turn": self.turn_count
        })
        if role == "user":
            self.turn_count += 1
            # Auto-extract facts from user messages
            self.extract_facts(content)

    def extract_facts(self, message: str):
        """
        Extract key facts from a user message for long-term storage.
        Uses simple pattern matching to identify personal information.
        """
        message_lower = message.lower()

        # Extract name
        name_patterns = [
            r"(?:my name is|i'm|i am|call me|this is)\s+([A-Z][a-z]+)",
            r"(?:hi|hello|hey),?\s+(?:my name is|i'm|i am)\s+([A-Z][a-z]+)",
        ]
        for pattern in name_patterns:
            match = re.search(pattern, message, re.IGNORECASE)
            if match:
                name = match.group(1).capitalize()
                self.update_profile("name", name)
                self.key_facts.append(f"User's name is {name}")
                break

        # Extract interests
        interest_patterns = [
            r"(?:interested in|want to learn|learning about|curious about|"
            r"fascinated by|working on|studying)\s+(.+?)(?:\.|$|,)",
        ]
        for pattern in interest_patterns:
            match = re.search(pattern, message_lower)
            if match:
                interest = match.group(1).strip().rstrip('.')
                if len(interest) > 3 and len(interest) < 100:
                    current = self.user_profile.get("interests", [])
                    if isinstance(current, list):
                        current.append(interest)
                    else:
                        current = [interest]
                    self.update_profile("interests", current)
                    self.key_facts.append(f"User is interested in: {interest}")

        # Extract experience level
        experience_patterns = [
            (r"(?:i'm a |i am a |i'm |i am )?(beginner|newbie|just starting|new to)",
             "beginner"),
            (r"(?:i'm |i am |i have )?(intermediate|some experience|familiar with)",
             "intermediate"),
            (r"(?:i'm |i am |i have )?(advanced|expert|experienced|senior|years of experience)",
             "advanced"),
            (r"(?:i'm a |i am a )?(student|undergrad|grad student|phd)",
             "student"),
            (r"(?:i'm a |i am a )?(researcher|professor|scientist)",
             "researcher"),
            (r"(?:i'm a |i am a )?(developer|engineer|programmer|software)",
             "developer"),
        ]
        for pattern, level in experience_patterns:
            if re.search(pattern, message_lower):
                self.update_profile("experience_level", level)
                self.key_facts.append(f"User's experience level: {level}")
                break

        # Extract goals
        goal_patterns = [
            r"(?:i want to|i'd like to|my goal is to|hoping to|trying to)\s+(.+?)(?:\.|$|,)",
        ]
        for pattern in goal_patterns:
            match = re.search(pattern, message_lower)
            if match:
                goal = match.group(1).strip().rstrip('.')
                if len(goal) > 5:
                    self.update_profile("goal", goal)
                    self.key_facts.append(f"User's goal: {goal}")

    def update_profile(self, key: str, value):
        """Update user profile (e.g., name, interest area)."""
        self.user_profile[key] = value

    def get_relevant_history(self, query: str = "", max_turns: int = 5) -> List[Dict]:
        """Get recent relevant conversation history."""
        # Return the most recent turns (simple recency-based approach)
        recent = self.conversation_history[-(max_turns * 2):]
        return recent

    def get_profile_summary(self) -> str:
        """Get a human-readable summary of what we know about the user."""
        if not self.user_profile:
            return "No user information collected yet."

        parts = []
        if "name" in self.user_profile:
            parts.append(f"Name: {self.user_profile['name']}")
        if "interests" in self.user_profile:
            interests = self.user_profile["interests"]
            if isinstance(interests, list):
                parts.append(f"Interests: {', '.join(interests)}")
            else:
                parts.append(f"Interests: {interests}")
        if "experience_level" in self.user_profile:
            parts.append(f"Experience: {self.user_profile['experience_level']}")
        if "goal" in self.user_profile:
            parts.append(f"Goal: {self.user_profile['goal']}")

        return " | ".join(parts) if parts else "No user information collected yet."

    def get_state(self) -> Dict:
        """Return full memory state for visualization."""
        return {
            "user_profile": dict(self.user_profile),
            "turn_count": self.turn_count,
            "key_facts": list(self.key_facts),
            "history_length": len(self.conversation_history)
        }

In [ ]:
#@title 🎧 Narration: Test Memory
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_28_test_memory.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the memory system
test_memory = ConversationMemory()

# Simulate extracting facts from messages
test_messages = [
    "Hi, my name is Alex and I'm interested in diffusion models",
    "I'm a grad student working on computer vision",
    "I want to build generative models for medical imaging",
]

print("Testing Memory System")
print("=" * 70)

for msg in test_messages:
    test_memory.add_turn("user", msg)
    print(f"\nMessage: \"{msg}\"")
    print(f"Profile: {test_memory.get_profile_summary()}")
    print(f"Key facts: {test_memory.key_facts}")
    print("-" * 70)

# Clean up test
del test_memory
print("\n✅ Memory extraction working correctly!")

In [ ]:
#@title 🎧 Transition: Component4 Tools Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_29_component4_tools_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Component 4: Tool System

Tools let the chatbot take actions beyond just generating text. We build a registry that lets us define, describe, and execute tools dynamically.

In [ ]:
#@title 🎧 Code Walkthrough: Toolregistry Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_30_toolregistry_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class ToolRegistry:
    """
    Registry for chatbot tools. Each tool has a name, function,
    description, and parameter spec.
    """

    def __init__(self):
        self.tools = {}  # name -> {"func": ..., "description": ..., "params": ...}

    def register(self, name: str, func: Callable, description: str,
                 params: Optional[Dict] = None):
        """Register a tool with its function and description."""
        self.tools[name] = {
            "func": func,
            "description": description,
            "params": params or {}
        }
        print(f"  Registered tool: {name}")

    def get_tool_descriptions(self) -> str:
        """Get formatted descriptions for all tools (for the LLM prompt)."""
        descriptions = []
        for name, info in self.tools.items():
            param_str = ""
            if info["params"]:
                param_str = f" Parameters: {json.dumps(info['params'])}"
            descriptions.append(f"- {name}: {info['description']}{param_str}")
        return "\n".join(descriptions)

    def execute(self, tool_name: str, **kwargs) -> str:
        """Execute a tool and return its string result."""
        if tool_name not in self.tools:
            return f"Error: Unknown tool '{tool_name}'"
        try:
            result = self.tools[tool_name]["func"](**kwargs)
            return str(result)
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"

    def detect_tool_need(self, message: str) -> Optional[Tuple[str, Dict]]:
        """
        Simple heuristic to detect if a message needs a tool.
        Returns (tool_name, kwargs) or None.
        """
        message_lower = message.lower()

        # Course lookup triggers
        course_triggers = ["what courses", "which courses", "course list",
                           "available courses", "what do you offer",
                           "what topics", "what can i learn",
                           "other courses", "more courses"]
        if any(trigger in message_lower for trigger in course_triggers):
            # Try to extract a topic
            topic_match = re.search(
                r"(?:about|on|for|related to|in)\s+(.+?)(?:\?|$|\.)",
                message_lower
            )
            topic = topic_match.group(1).strip() if topic_match else ""
            return ("course_lookup", {"topic": topic})

        # Contact triggers
        contact_triggers = ["contact", "reach", "email", "get in touch",
                            "how to contact", "talk to someone"]
        if any(trigger in message_lower for trigger in contact_triggers):
            return ("contact_info", {})

        # FAQ triggers (broad — catches many questions)
        faq_triggers = ["how do i", "what is", "do i need", "can i",
                         "prerequisite", "requirement", "gpu", "laptop",
                         "free", "cost", "price", "get started"]
        if any(trigger in message_lower for trigger in faq_triggers):
            return ("faq_search", {"question": message})

        return None

In [ ]:
#@title 🎧 Listen: Define Tool Functions Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_31_define_tool_functions_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let's define the actual tool functions and register them.

In [ ]:
#@title 🎧 Code Walkthrough: Tool Functions And Register
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_32_tool_functions_and_register.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ---- Tool Functions ----

COURSE_CATALOG = {
    "reinforcement learning": {
        "title": "Reinforcement Learning & DeepSeek-R1",
        "description": "Learn RL from scratch — bandits, Q-learning, policy gradients, PPO, and how DeepSeek-R1 uses RL to improve reasoning.",
        "notebooks": 5,
        "prerequisites": "Python, basic probability"
    },
    "diffusion models": {
        "title": "Diffusion Models for Video Generation",
        "description": "Build diffusion models from first principles — noise schedules, UNets, video generation with factorized space-time attention.",
        "notebooks": 3,
        "prerequisites": "Python, PyTorch basics, linear algebra"
    },
    "world models": {
        "title": "World Action Models for Robotics",
        "description": "From Ha & Schmidhuber to GENIE2 — learn how robots build internal models of the world.",
        "notebooks": 6,
        "prerequisites": "Python, PyTorch, basic RL concepts"
    },
    "vla": {
        "title": "Vision-Language-Action Models",
        "description": "Build VLA models for autonomous driving — vision encoders, action tokenization, and diffusion action decoders.",
        "notebooks": 4,
        "prerequisites": "Python, PyTorch, transformers basics"
    },
    "parallelism": {
        "title": "5D Parallelism for GPU Programming",
        "description": "Data, tensor, pipeline, sequence, and expert parallelism — train models across thousands of GPUs.",
        "notebooks": 6,
        "prerequisites": "Python, basic distributed computing concepts"
    },
    "context engineering": {
        "title": "Context Engineering for LLMs",
        "description": "Master the art of what goes into the context window — token budgets, RAG, memory, tools, and the four strategies.",
        "notebooks": 5,
        "prerequisites": "Python, basic LLM familiarity"
    },
    "vision transformers": {
        "title": "Vision Transformers from Scratch",
        "description": "Build ViT from scratch — patch embeddings, self-attention on image patches, complete classification pipeline.",
        "notebooks": 3,
        "prerequisites": "Python, PyTorch, basic deep learning"
    },
    "diffusion llms": {
        "title": "Diffusion Language Models from Scratch",
        "description": "A new paradigm — generate text through iterative unmasking instead of autoregressive decoding.",
        "notebooks": 4,
        "prerequisites": "Python, PyTorch, transformer basics"
    }
}


def course_lookup(topic: str = "") -> str:
    """Look up available Vizuara courses, optionally filtered by topic."""
    topic_lower = topic.lower().strip()

    if topic_lower:
        # Find matching courses
        matches = []
        for key, course in COURSE_CATALOG.items():
            if (topic_lower in key or
                topic_lower in course["title"].lower() or
                topic_lower in course["description"].lower()):
                matches.append(course)

        if matches:
            result = f"Found {len(matches)} course(s) related to '{topic}':\n"
            for c in matches:
                result += (f"\n- {c['title']}: {c['description']} "
                          f"({c['notebooks']} notebooks, "
                          f"prerequisites: {c['prerequisites']})")
            return result
        else:
            result = f"No courses directly matching '{topic}'. Here are all available courses:\n"
            for c in COURSE_CATALOG.values():
                result += f"\n- {c['title']}: {c['description']}"
            return result
    else:
        result = f"Vizuara offers {len(COURSE_CATALOG)} courses:\n"
        for c in COURSE_CATALOG.values():
            result += (f"\n- {c['title']} ({c['notebooks']} notebooks)")
        return result


def contact_info() -> str:
    """Get Vizuara contact information."""
    return ("You can reach Vizuara through:\n"
            "- Website: vizuara.ai\n"
            "- Substack: vizuara.substack.com (comment on any article)\n"
            "- Articles and notebooks: vizuara.substack.com\n"
            "Vizuara encourages questions and discussion from all learners.")


def faq_search(question: str) -> str:
    """Search the FAQ knowledge base for an answer."""
    # Use the vector store to find relevant FAQ entries
    results = vector_store.search(question, top_k=2)
    if results and results[0]["score"] > 0.3:
        faq_text = "Relevant FAQ information:\n"
        for r in results:
            if r["score"] > 0.3:
                faq_text += f"\n[{r['metadata'].get('topic', 'FAQ')}]: {r['text']}"
        return faq_text
    return "No specific FAQ entry found for this question."


# Register all tools
tools = ToolRegistry()
print("Registering tools:")
tools.register("course_lookup", course_lookup,
               "Look up available Vizuara courses on a topic",
               {"topic": "string (optional) - topic to filter by"})
tools.register("contact_info", contact_info,
               "Get Vizuara contact information")
tools.register("faq_search", faq_search,
               "Search the FAQ knowledge base for an answer",
               {"question": "string - the question to search for"})

print(f"\n{len(tools.tools)} tools registered.")
print(f"\nTool descriptions for the LLM:\n{tools.get_tool_descriptions()}")

In [ ]:
#@title 🎧 Narration: Test Tools
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_33_test_tools.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test each tool
print("Tool Execution Tests")
print("=" * 70)

print("\n1. Course lookup (topic='diffusion'):")
print(tools.execute("course_lookup", topic="diffusion"))

print("\n2. Contact info:")
print(tools.execute("contact_info"))

print("\n3. FAQ search (question='Do I need a GPU?'):")
print(tools.execute("faq_search", question="Do I need a GPU?"))

print("\n✅ All tools working correctly!")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_34_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn — TODO 1: Improve Fact Extraction

The `extract_facts` method above uses basic regex patterns. Your task is to make it smarter.

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_35_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_56_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ============ TODO ============
# Improve the ConversationMemory.extract_facts() method.
#
# Currently it catches simple patterns like "my name is Alex" and
# "I'm interested in diffusion models". Your job: add MORE patterns
# to extract richer user information.
#
# Specifically, implement extraction for:
#
# 1. OCCUPATION: Detect phrases like "I work as a ...", "I'm a data scientist",
#    "my job is ...", "I work at ..." and store in profile as "occupation".
#
# 2. PREFERRED LEARNING STYLE: Detect "I prefer ...", "I learn best by ...",
#    "I like hands-on ...", "I'm more of a visual learner" etc.
#    Store in profile as "learning_style".
#
# 3. PRIOR KNOWLEDGE: Detect "I already know ...", "I've used ...",
#    "I'm familiar with ...", "I've worked with ..." and store as
#    a list in profile["prior_knowledge"].
#
# Test your implementation with the test messages below.

class ImprovedConversationMemory(ConversationMemory):
    """Extended memory with better fact extraction."""

    def extract_facts(self, message: str):
        """Extract key facts including occupation, learning style, prior knowledge."""
        # First, call the parent's extraction for name, interests, etc.
        super().extract_facts(message)

        message_lower = message.lower()

        # --- YOUR CODE HERE ---

        # 1. OCCUPATION extraction
        # Hint: Look for patterns like "I work as a", "I'm a data scientist",
        #       "my job is", "I work at"
        occupation_patterns = [
            # Add your regex patterns here
            # Example: r"(?:i work as a|i'm a|i am a)\s+(.+?)(?:\.|$|,| and)"
        ]
        # for pattern in occupation_patterns:
        #     match = re.search(pattern, message_lower)
        #     if match:
        #         self.update_profile("occupation", match.group(1).strip())
        #         self.key_facts.append(f"User works as: {match.group(1).strip()}")
        #         break

        # 2. LEARNING STYLE extraction
        # Hint: Look for "I prefer", "I learn best", "I'm a visual/hands-on learner"
        pass

        # 3. PRIOR KNOWLEDGE extraction
        # Hint: Look for "I already know", "I've used", "I'm familiar with"
        # Store as a list in profile["prior_knowledge"]
        pass

        # --- END YOUR CODE ---


# Test your implementation
test_memory_2 = ImprovedConversationMemory()
test_messages_2 = [
    "Hi, my name is Sarah and I work as a data scientist at Google",
    "I learn best by doing — hands-on coding is my thing",
    "I already know PyTorch and I've used transformers before",
    "I'm interested in learning about diffusion models for my research",
]

print("Testing Improved Memory Extraction")
print("=" * 70)
for msg in test_messages_2:
    test_memory_2.add_turn("user", msg)
    print(f"\nMessage: \"{msg}\"")
    print(f"Profile: {test_memory_2.user_profile}")
    print("-" * 70)

print(f"\nAll key facts: {test_memory_2.key_facts}")

# Expected output should include:
# - name: Sarah
# - occupation: data scientist
# - learning_style: hands-on / learning by doing
# - prior_knowledge: [PyTorch, transformers]
# - interests: [diffusion models]

del test_memory_2

In [ ]:
#@title 🎧 Listen: Todo1 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_36_todo1_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Transition: Component5 Assembler Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_37_component5_assembler_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Component 5: Context Assembler

This is the heart of the system — where all four notebooks come together. The Context Assembler takes the user's query plus all the components (memory, RAG results, tool outputs) and assembles them into an optimal prompt that fits within the token budget.

It applies the techniques from every notebook:
- **Notebook 1**: Token budget calculation and allocation
- **Notebook 2**: Health checks for context failure modes
- **Notebook 3**: RAG retrieval and relevance scoring
- **Notebook 4**: The four strategies — write, select, compress, isolate

In [ ]:
#@title 🎧 Code Walkthrough: Assembler Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_38_assembler_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class ContextAssembler:
    """
    Assembles optimal context for the LLM within a token budget.
    Uses all four strategies from Notebooks 1-4.
    """

    def __init__(self, max_tokens: int = 30000):
        self.max_tokens = max_tokens
        self.assembly_log = []  # Track what went into the context

    def estimate_tokens(self, text: str) -> int:
        """Rough token estimation: ~4 characters per token for English."""
        return len(text) // 4

    def compress_text(self, text: str, target_ratio: float = 0.6) -> str:
        """
        Strategy: COMPRESS — reduce text while preserving key information.
        Simple extractive compression: keep the most information-dense sentences.
        """
        sentences = re.split(r'(?<=[.!?])\s+', text)
        if len(sentences) <= 2:
            return text

        # Score sentences by information density (length + unique words)
        scored = []
        for s in sentences:
            words = set(s.lower().split())
            # Prefer longer sentences with more unique words
            score = len(words) * 0.7 + len(s) * 0.01
            scored.append((score, s))

        scored.sort(reverse=True)

        # Keep top sentences until we hit target ratio
        target_len = int(len(text) * target_ratio)
        kept = []
        current_len = 0
        for score, s in scored:
            if current_len + len(s) <= target_len:
                kept.append(s)
                current_len += len(s)

        # Re-order by original position
        ordered = [s for s in sentences if s in kept]
        return " ".join(ordered) if ordered else text

    def health_check(self, context_parts: Dict[str, str]) -> List[str]:
        """
        Strategy: Health check from Notebook 2.
        Detect potential failure modes in the assembled context.
        """
        warnings_list = []

        # Check for stale context (no recent conversation history)
        if "history" in context_parts:
            if len(context_parts["history"]) < 10:
                warnings_list.append("SPARSE_HISTORY: Very little conversation context")

        # Check for conflicting information
        all_text = " ".join(context_parts.values()).lower()
        if "free" in all_text and "paid" in all_text:
            warnings_list.append("POTENTIAL_CONFLICT: Both 'free' and 'paid' mentioned")

        # Check for missing critical context
        if "retrieved" not in context_parts or not context_parts["retrieved"]:
            warnings_list.append("NO_RETRIEVAL: No RAG results included")

        # Check for context overflow risk
        total_tokens = sum(self.estimate_tokens(v) for v in context_parts.values())
        if total_tokens > self.max_tokens * 0.9:
            warnings_list.append(f"NEAR_OVERFLOW: {total_tokens}/{self.max_tokens} tokens used")

        return warnings_list

    def assemble(self, query: str, system_prompt: str,
                 memory: ConversationMemory,
                 retrieved_docs: List[Dict],
                 tool_results: Optional[str] = None) -> Tuple[str, Dict]:
        """
        Assemble optimal context within token budget.
        Returns (assembled_prompt, assembly_metadata).
        """
        self.assembly_log = []
        budget = self.max_tokens

        # ---- STRATEGY: WRITE (Notebook 4) ----
        # Craft a dynamic system prompt that includes user-specific context
        profile_summary = memory.get_profile_summary()
        dynamic_system = system_prompt
        if profile_summary != "No user information collected yet.":
            dynamic_system += f"\n\nUser profile: {profile_summary}"
        system_tokens = self.estimate_tokens(dynamic_system)
        budget -= system_tokens
        self.assembly_log.append(f"System prompt: {system_tokens} tokens")

        # ---- STRATEGY: SELECT (Notebook 4) ----
        # Select the most relevant conversation history
        history = memory.get_relevant_history(query, max_turns=5)
        history_text = ""
        if history:
            history_parts = []
            for turn in history:
                role_label = "User" if turn["role"] == "user" else "Assistant"
                history_parts.append(f"{role_label}: {turn['content']}")
            history_text = "\n".join(history_parts)

            history_tokens = self.estimate_tokens(history_text)
            # If history is too long, compress it
            if history_tokens > budget * 0.3:
                history_text = self.compress_text(history_text, target_ratio=0.5)
                history_tokens = self.estimate_tokens(history_text)
                self.assembly_log.append(
                    f"History: {history_tokens} tokens (COMPRESSED)")
            else:
                self.assembly_log.append(f"History: {history_tokens} tokens")
            budget -= history_tokens

        # ---- STRATEGY: SELECT + COMPRESS (Notebooks 3 & 4) ----
        # Include retrieved documents, prioritized by relevance score
        retrieved_text = ""
        if retrieved_docs:
            doc_parts = []
            for i, doc in enumerate(retrieved_docs):
                if doc["score"] > 0.25:  # Relevance threshold
                    text = doc["text"]
                    doc_tokens = self.estimate_tokens(text)
                    if doc_tokens > budget * 0.15:
                        text = self.compress_text(text, target_ratio=0.5)
                    doc_parts.append(
                        f"[Source: {doc['metadata'].get('topic', 'unknown')} | "
                        f"Relevance: {doc['score']:.2f}]\n{text}")
            retrieved_text = "\n\n".join(doc_parts)
            ret_tokens = self.estimate_tokens(retrieved_text)
            budget -= ret_tokens
            self.assembly_log.append(
                f"Retrieved docs: {ret_tokens} tokens "
                f"({len(doc_parts)} docs)")

        # ---- STRATEGY: ISOLATE (Notebook 4) ----
        # Tool results get their own clearly-delimited section
        tool_text = ""
        if tool_results:
            tool_text = f"[TOOL RESULTS]\n{tool_results}\n[/TOOL RESULTS]"
            tool_tokens = self.estimate_tokens(tool_text)
            budget -= tool_tokens
            self.assembly_log.append(f"Tool results: {tool_tokens} tokens")

        # ---- HEALTH CHECK (Notebook 2) ----
        context_parts = {
            "system": dynamic_system,
            "history": history_text,
            "retrieved": retrieved_text,
            "tools": tool_text,
            "query": query,
        }
        warnings_list = self.health_check(context_parts)
        if warnings_list:
            self.assembly_log.append(f"Health warnings: {warnings_list}")

        # ---- FINAL ASSEMBLY ----
        # Build the complete prompt with clear section delimiters
        sections = [dynamic_system]

        if history_text:
            sections.append(f"\n--- Conversation History ---\n{history_text}")

        if retrieved_text:
            sections.append(
                f"\n--- Relevant Knowledge ---\n{retrieved_text}")

        if tool_text:
            sections.append(f"\n{tool_text}")

        if memory.key_facts:
            facts_text = "\n".join(f"- {f}" for f in memory.key_facts[-5:])
            sections.append(
                f"\n--- Key Facts About This User ---\n{facts_text}")

        sections.append(f"\n--- Current Question ---\n{query}")

        assembled = "\n".join(sections)

        # Token budget report
        total_tokens = self.estimate_tokens(assembled)
        metadata = {
            "total_tokens": total_tokens,
            "max_tokens": self.max_tokens,
            "utilization": total_tokens / self.max_tokens,
            "assembly_log": self.assembly_log,
            "warnings": warnings_list,
            "sections": {
                "system": system_tokens,
                "history": self.estimate_tokens(history_text),
                "retrieved": self.estimate_tokens(retrieved_text),
                "tools": self.estimate_tokens(tool_text),
                "query": self.estimate_tokens(query),
            }
        }

        return assembled, metadata

In [ ]:
#@title 🎧 Narration: Test Assembler
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_39_test_assembler.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the context assembler with a mock scenario
test_assembler = ContextAssembler(max_tokens=30000)
test_mem = ConversationMemory()
test_mem.add_turn("user", "Hi, my name is Alex and I'm interested in diffusion models")
test_mem.add_turn("assistant", "Hi Alex! Great to hear you're interested in diffusion models.")

test_system = ("You are a friendly assistant for Vizuara AI Labs. "
               "Help students learn about courses and AI concepts.")

test_retrieved = vector_store.search("diffusion models course", top_k=3)

assembled, meta = test_assembler.assemble(
    query="What prerequisites do I need for the diffusion models course?",
    system_prompt=test_system,
    memory=test_mem,
    retrieved_docs=test_retrieved,
    tool_results=None
)

print("Context Assembly Report")
print("=" * 70)
print(f"Total tokens: {meta['total_tokens']} / {meta['max_tokens']}")
print(f"Utilization: {meta['utilization']:.1%}")
print(f"\nSection breakdown:")
for section, tokens in meta['sections'].items():
    bar = "█" * (tokens // 5) if tokens > 0 else ""
    print(f"  {section:12s}: {tokens:5d} tokens {bar}")
print(f"\nAssembly log:")
for entry in meta['assembly_log']:
    print(f"  - {entry}")
if meta['warnings']:
    print(f"\nHealth warnings:")
    for w in meta['warnings']:
        print(f"  ⚠ {w}")

print(f"\n--- First 500 chars of assembled context ---")
print(assembled[:500])

del test_assembler, test_mem

In [ ]:
#@title 🎧 What to Look For: Viz2 Token Budget Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_40_viz2_token_budget_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Visualization Checkpoint 2: Token Budget Allocation

Let's visualize how the context assembler allocates the token budget across different sections. This is the Notebook 1 technique in action.

In [ ]:
#@title 🎧 What to Look For: Viz2 Token Budget Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_41_viz2_token_budget_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualization: Token budget allocation as a stacked bar + pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

sections = meta['sections']
section_names = list(sections.keys())
section_values = list(sections.values())
colors = ['#1976D2', '#7B1FA2', '#388E3C', '#D32F2F', '#F57C00']

# Stacked horizontal bar
left = 0
for i, (name, val) in enumerate(zip(section_names, section_values)):
    ax1.barh(0, val, left=left, color=colors[i], edgecolor='white',
             height=0.6, label=f"{name} ({val} tokens)")
    if val > 20:
        ax1.text(left + val/2, 0, f"{name}\n{val}",
                 ha='center', va='center', fontsize=8,
                 fontweight='bold', color='white')
    left += val

# Show remaining budget
remaining = meta['max_tokens'] - sum(section_values)
ax1.barh(0, remaining, left=left, color='#EEEEEE', edgecolor='white',
         height=0.6, label=f"remaining ({remaining} tokens)")
ax1.text(left + remaining/2, 0, f"available\n{remaining}",
         ha='center', va='center', fontsize=8, color='#999')

ax1.set_xlim(0, meta['max_tokens'])
ax1.set_yticks([])
ax1.set_xlabel("Tokens", fontsize=11)
ax1.set_title("Token Budget Allocation (Notebook 1 Technique)",
              fontsize=12, fontweight='bold')
ax1.legend(loc='upper right', fontsize=8)

# Pie chart of used tokens
used_names = [n for n, v in zip(section_names, section_values) if v > 0]
used_values = [v for v in section_values if v > 0]
used_colors = [c for c, v in zip(colors, section_values) if v > 0]

ax2.pie(used_values, labels=used_names, colors=used_colors,
        autopct='%1.0f%%', startangle=90, textprops={'fontsize': 10})
ax2.set_title(f"Context Composition\n({sum(used_values)} tokens used)",
              fontsize=12, fontweight='bold')

plt.suptitle("How the Context Window Is Filled",
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Component6 Chatbot Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_42_component6_chatbot_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.6 Component 6: The Chatbot

Now we tie everything together. The `VizuaraChatbot` class orchestrates all components into a single, coherent conversational agent.

In [ ]:
#@title 🎧 Code Walkthrough: Chatbot Class Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_43_chatbot_class_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
CHATBOT_SYSTEM_PROMPT = """You are a friendly, knowledgeable assistant for Vizuara AI Labs. You help students learn about Vizuara's courses, articles, and teaching philosophy.

Key behaviors:
- Be warm and encouraging — students are here to learn
- If you know the user's name, use it naturally in your responses
- Reference the user's interests when suggesting content
- If asked about something not in your knowledge base, say so honestly
- Always encourage hands-on learning with the Colab notebooks
- Keep responses concise but helpful (2-4 sentences for simple questions, more for complex ones)
- When recommending courses, mention the number of notebooks and prerequisites
- If the user seems new, be extra welcoming and suggest starting points"""


class VizuaraChatbot:
    """
    The complete FAQ chatbot — ties together knowledge base, vector store,
    memory, tools, and context assembler into a conversational agent.
    """

    def __init__(self, api_key_configured: bool = True):
        # Initialize all components
        self.knowledge_base = kb           # Already built above
        self.vector_store = vector_store   # Already built above
        self.memory = ConversationMemory()
        self.tools = tools                 # Already built above
        self.assembler = ContextAssembler(max_tokens=30000)
        self.model = genai.GenerativeModel(MODEL_NAME)
        self.system_prompt = CHATBOT_SYSTEM_PROMPT

        # Conversation metadata for visualization
        self.turn_metadata = []

    def chat(self, user_message: str) -> str:
        """
        Process a user message and generate a response.
        This is the main entry point — it orchestrates all components.
        """
        turn_info = {
            "turn": self.memory.turn_count + 1,
            "user_message": user_message,
            "tools_called": [],
            "docs_retrieved": [],
            "memory_updates": [],
            "token_usage": {}
        }

        # Step 1: Update memory with the user's message
        profile_before = dict(self.memory.user_profile)
        self.memory.add_turn("user", user_message)
        profile_after = dict(self.memory.user_profile)

        # Track what changed in memory
        for key in profile_after:
            if key not in profile_before or profile_after[key] != profile_before[key]:
                turn_info["memory_updates"].append(
                    f"{key}: {profile_after[key]}")

        # Step 2: Check if tools are needed
        tool_result = None
        tool_detection = self.tools.detect_tool_need(user_message)
        if tool_detection:
            tool_name, tool_kwargs = tool_detection
            tool_result = self.tools.execute(tool_name, **tool_kwargs)
            turn_info["tools_called"].append({
                "name": tool_name,
                "args": tool_kwargs,
                "result_preview": tool_result[:100] + "..."
                                   if len(tool_result) > 100 else tool_result
            })

        # Step 3: Retrieve relevant documents
        retrieved = self.vector_store.search(user_message, top_k=3)
        turn_info["docs_retrieved"] = [
            {"topic": r["metadata"].get("topic", "unknown"),
             "score": round(r["score"], 3)}
            for r in retrieved
        ]

        # Step 4: Assemble context (the core of context engineering!)
        assembled_context, assembly_meta = self.assembler.assemble(
            query=user_message,
            system_prompt=self.system_prompt,
            memory=self.memory,
            retrieved_docs=retrieved,
            tool_results=tool_result
        )
        turn_info["token_usage"] = assembly_meta

        # Step 5: Generate response via Gemini
        try:
            response = self.model.generate_content(assembled_context)
            assistant_message = response.text.strip()
        except Exception as e:
            assistant_message = (
                f"I apologize, but I encountered an error generating a response. "
                f"Please try again. (Error: {str(e)[:100]})")

        # Step 6: Update conversation history with the response
        self.memory.add_turn("assistant", assistant_message)

        # Store metadata for this turn
        self.turn_metadata.append(turn_info)

        return assistant_message

    def get_turn_report(self, turn_index: int = -1) -> Dict:
        """Get detailed metadata for a specific turn."""
        if not self.turn_metadata:
            return {}
        return self.turn_metadata[turn_index]

    def get_memory_state(self) -> Dict:
        """Get the current memory state."""
        return self.memory.get_state()

    def visualize_turn(self, turn_index: int = -1):
        """Visualize what happened during a specific turn."""
        if not self.turn_metadata:
            print("No turns to visualize yet.")
            return

        info = self.turn_metadata[turn_index]
        print(f"\n{'=' * 70}")
        print(f"  Turn {info['turn']} Analysis")
        print(f"{'=' * 70}")

        print(f"\n  User: \"{info['user_message'][:80]}...\""
              if len(info['user_message']) > 80
              else f"\n  User: \"{info['user_message']}\"")

        # Memory updates
        if info['memory_updates']:
            print(f"\n  Memory Updates:")
            for update in info['memory_updates']:
                print(f"    + {update}")
        else:
            print(f"\n  Memory Updates: (none)")

        # Tools called
        if info['tools_called']:
            print(f"\n  Tools Called:")
            for tool in info['tools_called']:
                print(f"    -> {tool['name']}({tool['args']})")
                print(f"       Result: {tool['result_preview']}")
        else:
            print(f"\n  Tools Called: (none)")

        # Documents retrieved
        if info['docs_retrieved']:
            print(f"\n  RAG Retrieved:")
            for doc in info['docs_retrieved']:
                score_bar = "█" * int(doc['score'] * 20)
                print(f"    [{doc['score']:.3f}] {score_bar} {doc['topic']}")
        else:
            print(f"\n  RAG Retrieved: (none)")

        # Token usage
        usage = info['token_usage']
        if usage:
            print(f"\n  Token Budget:")
            total = usage.get('total_tokens', 0)
            max_t = usage.get('max_tokens', 30000)
            pct = usage.get('utilization', 0)
            bar_len = int(pct * 40)
            bar = "█" * bar_len + "░" * (40 - bar_len)
            print(f"    [{bar}] {total}/{max_t} ({pct:.1%})")

        print(f"\n{'=' * 70}")

In [ ]:
#@title 🎧 Narration: Build Chatbot
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_44_build_chatbot.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Build the chatbot!
print("Initializing Vizuara FAQ Chatbot...")
print("-" * 40)
chatbot = VizuaraChatbot()
print("-" * 40)
print("\n✅ Chatbot ready! All components connected:")
print(f"   Knowledge Base: {len(kb.chunks)} chunks")
print(f"   Vector Store:   {vector_store.get_stats()['num_entries']} embeddings")
print(f"   Tools:          {len(tools.tools)} tools registered")
print(f"   Model:          {MODEL_NAME}")
print(f"   Max tokens:     {chatbot.assembler.max_tokens}")

In [ ]:
#@title 🎧 Transition: Live Demo Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_45_live_demo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together — The Live Demo

This is the moment of truth. We run a multi-turn conversation that demonstrates every feature: memory, RAG, tools, and context assembly working together.

Watch what happens at each turn — the chatbot remembers who the user is, retrieves relevant knowledge, calls tools when needed, and personalizes every response.

In [ ]:
#@title 🎧 Narration: Demo Turn1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_46_demo_turn1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Turn 1: Introduction with personal info
print("╔" + "═" * 68 + "╗")
print("║  LIVE DEMO: Multi-Turn Conversation with the Vizuara Chatbot    ║")
print("╚" + "═" * 68 + "╝")

user_msg_1 = "Hi, my name is Alex and I'm interested in learning about diffusion models"

print(f"\n👤 User: {user_msg_1}")
response_1 = chatbot.chat(user_msg_1)
print(f"\n🤖 Chatbot: {response_1}")
chatbot.visualize_turn(-1)

In [ ]:
#@title 🎧 Narration: Demo Turn2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_47_demo_turn2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Turn 2: Follow-up that requires memory + RAG
user_msg_2 = "What prerequisites do I need?"

print(f"\n👤 User: {user_msg_2}")
response_2 = chatbot.chat(user_msg_2)
print(f"\n🤖 Chatbot: {response_2}")
chatbot.visualize_turn(-1)

Notice how the chatbot remembers that Alex asked about diffusion models — even though Turn 2 does not mention diffusion models at all. The conversation history provides the context the model needs to give a relevant answer.

In [ ]:
#@title 🎧 Narration: Demo Turn3
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_48_demo_turn3.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Turn 3: GPU/hardware question (triggers FAQ search)
user_msg_3 = "Can I run the notebooks on my laptop?"

print(f"\n👤 User: {user_msg_3}")
response_3 = chatbot.chat(user_msg_3)
print(f"\n🤖 Chatbot: {response_3}")
chatbot.visualize_turn(-1)

In [ ]:
#@title 🎧 Narration: Demo Turn4
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_49_demo_turn4.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Turn 4: Course discovery (triggers course_lookup tool)
user_msg_4 = "What other courses do you have?"

print(f"\n👤 User: {user_msg_4}")
response_4 = chatbot.chat(user_msg_4)
print(f"\n🤖 Chatbot: {response_4}")
chatbot.visualize_turn(-1)

In [ ]:
#@title 🎧 Narration: Demo Turn5
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_50_demo_turn5.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Turn 5: Teaching philosophy (personalized retrieval)
user_msg_5 = "Tell me more about the teaching philosophy"

print(f"\n👤 User: {user_msg_5}")
response_5 = chatbot.chat(user_msg_5)
print(f"\n🤖 Chatbot: {response_5}")
chatbot.visualize_turn(-1)

In [ ]:
#@title 🎧 What to Look For: Viz3 Dashboard Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_51_viz3_dashboard_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Visualization Checkpoint 3: Conversation Dashboard

Now let's build a comprehensive dashboard showing everything that happened across all 5 turns — memory evolution, tool usage, retrieval scores, and token consumption.

In [ ]:
#@title 🎧 What to Look For: Viz3 Dashboard Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_52_viz3_dashboard_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Full conversation dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

turns = list(range(1, len(chatbot.turn_metadata) + 1))

# ---- Panel 1: Token usage per turn ----
ax = axes[0, 0]
token_totals = []
section_data = {
    "system": [], "history": [], "retrieved": [], "tools": [], "query": []
}

for info in chatbot.turn_metadata:
    usage = info.get("token_usage", {})
    sections = usage.get("sections", {})
    total = usage.get("total_tokens", 0)
    token_totals.append(total)
    for key in section_data:
        section_data[key].append(sections.get(key, 0))

# Stacked bar chart
bottom = np.zeros(len(turns))
colors_sections = ['#1976D2', '#7B1FA2', '#388E3C', '#D32F2F', '#F57C00']
for i, (key, values) in enumerate(section_data.items()):
    ax.bar(turns, values, bottom=bottom, label=key,
           color=colors_sections[i], edgecolor='white', width=0.6)
    bottom += np.array(values)

ax.set_xlabel("Turn", fontsize=11)
ax.set_ylabel("Tokens", fontsize=11)
ax.set_title("Token Usage Per Turn", fontsize=13, fontweight='bold')
ax.legend(fontsize=9, loc='upper left')
ax.set_xticks(turns)

# ---- Panel 2: RAG retrieval scores ----
ax = axes[0, 1]
for i, info in enumerate(chatbot.turn_metadata):
    docs = info.get("docs_retrieved", [])
    scores = [d["score"] for d in docs]
    topics = [d["topic"][:15] for d in docs]
    x_positions = [turns[i] - 0.2, turns[i], turns[i] + 0.2][:len(scores)]
    colors_rag = ['#4CAF50', '#81C784', '#C8E6C9'][:len(scores)]
    bars = ax.bar(x_positions, scores, width=0.18, color=colors_rag,
                  edgecolor='white')

ax.axhline(y=0.25, color='red', linestyle='--', alpha=0.5,
           label='Relevance threshold')
ax.set_xlabel("Turn", fontsize=11)
ax.set_ylabel("Cosine Similarity", fontsize=11)
ax.set_title("RAG Retrieval Scores", fontsize=13, fontweight='bold')
ax.set_xticks(turns)
ax.set_ylim(0, 1)
ax.legend(fontsize=9)

# ---- Panel 3: Memory evolution ----
ax = axes[1, 0]
profile_keys_over_time = []
facts_over_time = []

# Replay memory evolution
replay_mem = ConversationMemory()
for info in chatbot.turn_metadata:
    replay_mem.add_turn("user", info["user_message"])
    profile_keys_over_time.append(len(replay_mem.user_profile))
    facts_over_time.append(len(replay_mem.key_facts))

ax.plot(turns, profile_keys_over_time, 'o-', color='#9C27B0',
        linewidth=2, markersize=8, label='Profile fields')
ax.plot(turns, facts_over_time, 's-', color='#FF9800',
        linewidth=2, markersize=8, label='Key facts stored')
ax.fill_between(turns, profile_keys_over_time, alpha=0.1, color='#9C27B0')
ax.fill_between(turns, facts_over_time, alpha=0.1, color='#FF9800')
ax.set_xlabel("Turn", fontsize=11)
ax.set_ylabel("Count", fontsize=11)
ax.set_title("Memory Growth Over Conversation", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.set_xticks(turns)

# ---- Panel 4: Tool calls per turn ----
ax = axes[1, 1]
tool_calls = [len(info.get("tools_called", [])) for info in chatbot.turn_metadata]
tool_names = []
for info in chatbot.turn_metadata:
    called = info.get("tools_called", [])
    if called:
        tool_names.append(called[0]["name"])
    else:
        tool_names.append("(none)")

bar_colors_tools = ['#E0E0E0' if t == 0 else '#F44336' for t in tool_calls]
bars = ax.bar(turns, tool_calls, color=bar_colors_tools,
              edgecolor='white', width=0.6)

for i, (count, name) in enumerate(zip(tool_calls, tool_names)):
    ax.text(turns[i], count + 0.05, name, ha='center', va='bottom',
            fontsize=8, rotation=30, color='#333')

ax.set_xlabel("Turn", fontsize=11)
ax.set_ylabel("Tools Called", fontsize=11)
ax.set_title("Tool Usage Per Turn", fontsize=13, fontweight='bold')
ax.set_xticks(turns)
ax.set_ylim(0, max(tool_calls) + 1 if max(tool_calls) > 0 else 2)

plt.suptitle("Conversation Dashboard — All Systems Working Together",
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Memory State Summary Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_53_memory_state_summary_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let's also look at the full memory state after all 5 turns.

In [ ]:
#@title 🎧 Narration: Memory State Summary Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_54_memory_state_summary_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Memory state summary
state = chatbot.get_memory_state()

print("Memory State After 5 Turns")
print("=" * 70)
print(f"\n  User Profile:")
for key, value in state["user_profile"].items():
    print(f"    {key}: {value}")
print(f"\n  Key Facts ({len(state['key_facts'])}):")
for fact in state["key_facts"]:
    print(f"    - {fact}")
print(f"\n  Conversation turns: {state['turn_count']}")
print(f"  History entries: {state['history_length']}")
print(f"\n  Profile summary: {chatbot.memory.get_profile_summary()}")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_55_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Your Turn — TODO 2: Build a Learning Path Recommender

Now you build a new tool and register it with the chatbot. This is the capstone exercise — you are extending a working system with a new capability.

In [ ]:
# ============ TODO ============
# Implement a learning path recommender tool and register it.
#
# The tool should:
# 1. Take a user's interests and experience level as input
# 2. Recommend a sequence of 3-5 Vizuara courses in a logical order
# 3. Explain WHY each course is recommended and how they build on each other
#
# Use the COURSE_CATALOG dictionary defined earlier.
#
# Hint: Think about prerequisite chains. For example:
#   - A beginner interested in generative AI might start with
#     Vision Transformers → Diffusion Models → Diffusion LLMs
#   - Someone interested in robotics might go:
#     World Models → VLA Models
#   - Someone interested in systems might start with:
#     5D Parallelism → Context Engineering

def recommend_learning_path(interests: str = "",
                             experience_level: str = "beginner") -> str:
    """
    Recommend a personalized learning path through Vizuara courses.

    Args:
        interests: comma-separated list of user interests
        experience_level: beginner, intermediate, or advanced

    Returns:
        A formatted learning path recommendation string.
    """
    # --- YOUR CODE HERE ---
    #
    # Step 1: Parse the interests string into a list of topics
    # Step 2: Match interests to courses in COURSE_CATALOG
    # Step 3: Order courses logically (foundational → advanced)
    # Step 4: If experience_level is "advanced", skip intro courses
    # Step 5: Format and return a numbered learning path
    #
    # Example output:
    # "Recommended Learning Path for 'generative AI' (beginner):
    #
    #  1. Vision Transformers from Scratch (3 notebooks)
    #     → Start here to understand attention and patches
    #
    #  2. Diffusion Models for Video Generation (3 notebooks)
    #     → Build on transformer knowledge to generate images and video
    #
    #  3. Diffusion Language Models from Scratch (4 notebooks)
    #     → Apply diffusion to text — a new generation paradigm
    #
    #  Total: 10 notebooks, ~5-7 hours of hands-on learning"

    result = f"Recommended Learning Path (experience: {experience_level}):\n\n"

    # Placeholder — replace with your implementation
    result += "(TODO: Implement learning path logic here)\n"
    result += "\nHint: Use COURSE_CATALOG to find matching courses,\n"
    result += "then order them from foundational to advanced."

    return result
    # --- END YOUR CODE ---


# Register it with the tool registry
# (Uncomment and complete after implementing)
# chatbot.tools.register(
#     "recommend_learning_path",
#     recommend_learning_path,
#     "Recommend a personalized sequence of Vizuara courses based on "
#     "interests and experience level",
#     {"interests": "string - user's interests",
#      "experience_level": "string - beginner/intermediate/advanced"}
# )

# Test your implementation:
# print(recommend_learning_path("generative AI, diffusion models", "beginner"))
# print(recommend_learning_path("robotics, autonomous driving", "intermediate"))

In [ ]:
#@title 🎧 Listen: Todo2 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_57_todo2_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Viz Context Window Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_58_viz_context_window_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Putting It All Together — Full Context Window Visualization

Let's build one final visualization: a detailed view of exactly what is inside the context window for a single turn. This is the ultimate diagnostic — you can see every piece of context that influenced the model's response.

In [ ]:
#@title 🎧 What to Look For: Viz Context Window Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_59_viz_context_window_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Pick the most interesting turn (Turn 4: course lookup with tools)
turn_idx = 3  # 0-indexed (Turn 4)
info = chatbot.turn_metadata[turn_idx]
usage = info.get("token_usage", {})
sections = usage.get("sections", {})

fig, ax = plt.subplots(1, 1, figsize=(14, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title(f"Inside the Context Window — Turn {turn_idx + 1}",
             fontsize=16, fontweight='bold', pad=20)

# Draw the context window as a tall rectangle
context_rect = plt.Rectangle((0.5, 0.5), 9, 11, linewidth=3,
                                edgecolor='#333', facecolor='#FAFAFA',
                                linestyle='-')
ax.add_patch(context_rect)
ax.text(5, 11.7, f"Context Window ({usage.get('total_tokens', 0)} / "
        f"{usage.get('max_tokens', 30000)} tokens)",
        ha='center', fontsize=12, fontweight='bold', color='#333')

# Draw sections as colored blocks inside the context window
section_config = [
    ("System Prompt", '#1976D2', '#E3F2FD', sections.get('system', 0),
     "Personality + user profile"),
    ("Conversation History", '#7B1FA2', '#F3E5F5', sections.get('history', 0),
     "Recent turns for continuity"),
    ("Retrieved Knowledge (RAG)", '#388E3C', '#E8F5E9', sections.get('retrieved', 0),
     "Top-K relevant chunks"),
    ("Tool Results", '#D32F2F', '#FFEBEE', sections.get('tools', 0),
     "Course catalog, FAQ data"),
    ("User Query", '#F57C00', '#FFF3E0', sections.get('query', 0),
     "The actual question"),
]

total_used = sum(s[3] for s in section_config)
y_pos = 10.5
for name, edge, face, tokens, desc in section_config:
    if tokens == 0:
        continue
    # Height proportional to token usage
    height = max(0.5, (tokens / max(total_used, 1)) * 8)
    rect = plt.Rectangle((1.0, y_pos - height), 8, height,
                           linewidth=2, edgecolor=edge, facecolor=face,
                           alpha=0.8)
    ax.add_patch(rect)
    ax.text(5, y_pos - height/2, f"{name}\n{tokens} tokens",
            ha='center', va='center', fontsize=10, fontweight='bold',
            color=edge)
    ax.text(9.3, y_pos - height/2, desc, ha='left', va='center',
            fontsize=8, color='#666', style='italic')
    y_pos -= height + 0.15

# Show remaining budget
remaining = usage.get('max_tokens', 30000) - total_used
if remaining > 0:
    remaining_height = max(0.3, (remaining / usage.get('max_tokens', 30000)) * 8)
    remaining_height = min(remaining_height, y_pos - 0.7)
    if remaining_height > 0:
        rect = plt.Rectangle((1.0, y_pos - remaining_height), 8,
                               remaining_height, linewidth=1,
                               edgecolor='#CCC', facecolor='#F5F5F5',
                               linestyle='--', alpha=0.5)
        ax.add_patch(rect)
        ax.text(5, y_pos - remaining_height/2, f"Available: {remaining} tokens",
                ha='center', va='center', fontsize=9, color='#AAA')

# Annotations
ax.text(5, -0.3, "Each turn, the Context Assembler fills this window optimally",
        ha='center', fontsize=10, style='italic', color='#666')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Interactive Chat Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_60_interactive_chat_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Live Demo — Interactive Chat Mode

Let's run one more conversation to show the chatbot's full capabilities. This time we will add a few more turns and watch the memory grow.

In [ ]:
#@title 🎧 Narration: Interactive Chat Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_61_interactive_chat_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Additional conversation turns
additional_turns = [
    "I'm a beginner with Python but I know some math. Where should I start?",
    "That sounds great! How long will each notebook take?",
    "Can you remind me what my name is?"
]

print("╔" + "═" * 68 + "╗")
print("║  EXTENDED DEMO: More Turns to Show Memory Persistence           ║")
print("╚" + "═" * 68 + "╝")

for user_msg in additional_turns:
    print(f"\n👤 User: {user_msg}")
    response = chatbot.chat(user_msg)
    print(f"\n🤖 Chatbot: {response}")
    chatbot.visualize_turn(-1)
    print()

In [ ]:
#@title 🎧 What to Look For: Final Viz Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_62_final_viz_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Final Visualization: Complete Conversation Flow

This diagram shows the full conversation arc — every turn, every component that was activated, and how memory accumulated over the entire interaction.

In [ ]:
#@title 🎧 What to Look For: Final Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_63_final_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Timeline visualization of the full conversation
n_turns = len(chatbot.turn_metadata)

fig, ax = plt.subplots(1, 1, figsize=(16, max(8, n_turns * 1.2)))
ax.set_xlim(0, 10)
ax.set_ylim(0, n_turns + 1)
ax.axis('off')
ax.set_title("Complete Conversation Flow — Every Component in Action",
             fontsize=15, fontweight='bold', pad=20)

for i, info in enumerate(chatbot.turn_metadata):
    y = n_turns - i  # Top to bottom

    # Turn number
    ax.text(0.3, y, f"T{info['turn']}", fontsize=12, fontweight='bold',
            ha='center', va='center',
            bbox=dict(boxstyle='circle', facecolor='#1976D2', edgecolor='white'),
            color='white')

    # User message (truncated)
    msg = info['user_message'][:50]
    if len(info['user_message']) > 50:
        msg += "..."
    ax.text(1.0, y + 0.15, f"User: {msg}", fontsize=8.5, va='center',
            color='#333', fontweight='bold')

    # Component indicators
    x_start = 1.0
    indicators = []

    # Memory updates
    if info.get('memory_updates'):
        indicators.append(("MEM", '#9C27B0'))

    # Tools
    if info.get('tools_called'):
        tool_name = info['tools_called'][0]['name']
        indicators.append((f"TOOL:{tool_name}", '#D32F2F'))

    # RAG
    docs = info.get('docs_retrieved', [])
    high_score_docs = [d for d in docs if d['score'] > 0.3]
    if high_score_docs:
        indicators.append((f"RAG({len(high_score_docs)})", '#388E3C'))

    # Token usage
    total_tokens = info.get('token_usage', {}).get('total_tokens', 0)
    indicators.append((f"{total_tokens}tok", '#F57C00'))

    indicator_text = " | ".join(f"{label}" for label, _ in indicators)
    indicator_colors = [c for _, c in indicators]

    x_pos = 1.0
    for label, color in indicators:
        ax.text(x_pos, y - 0.2, label, fontsize=7, va='center',
                color='white', fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.2', facecolor=color,
                         edgecolor='none', alpha=0.8))
        x_pos += len(label) * 0.11 + 0.4

    # Connecting line
    if i < n_turns - 1:
        ax.plot([0.3, 0.3], [y - 0.4, y - 0.6], '-', color='#1976D2',
                linewidth=2, alpha=0.3)

# Legend at the bottom
legend_items = [
    ("MEM = Memory updated", '#9C27B0'),
    ("TOOL = Tool called", '#D32F2F'),
    ("RAG = Docs retrieved", '#388E3C'),
    ("tok = Tokens used", '#F57C00'),
]
for j, (label, color) in enumerate(legend_items):
    ax.text(0.5 + j * 2.5, -0.3, label, fontsize=8, va='center',
            color=color, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_64_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 10. Reflection

### What You Built

Let's take a step back and appreciate what just happened. You built a complete, working chatbot that uses **every** technique from this course:

| Notebook | Technique | Where It Shows Up |
|----------|-----------|------------------|
| **Notebook 1** | Token budget management | `ContextAssembler.assemble()` — explicit budget allocation per section |
| **Notebook 2** | Failure mode detection | `ContextAssembler.health_check()` — stale, conflicting, missing, overflow |
| **Notebook 3** | RAG retrieval | `VectorStore.search()` — cosine similarity over embedded chunks |
| **Notebook 4** | Write strategy | Dynamic system prompt with user profile injected |
| **Notebook 4** | Select strategy | `ConversationMemory.get_relevant_history()` — recent turns only |
| **Notebook 4** | Compress strategy | `ContextAssembler.compress_text()` — extractive compression |
| **Notebook 4** | Isolate strategy | Tool results in `[TOOL RESULTS]...[/TOOL RESULTS]` delimiters |

This is context engineering in practice. The model did not change between turns — only the **context** changed. And that context is what makes the chatbot feel intelligent, personalized, and useful.

In [ ]:
#@title 🎧 Listen: Reflection Questions
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_65_reflection_questions.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Reflection Questions

1. **Why does the chatbot feel "smart" even though it has no training on Vizuara data?** Because context engineering puts the right information in the right place at the right time. The model's intelligence comes from the context we assemble, not from fine-tuning.

2. **What would break if we removed the memory system?** The chatbot would forget the user's name by Turn 2. It would not know what "prerequisites" refers to (diffusion models). Every turn would be an isolated question with no continuity.

3. **Why do we compress conversation history instead of keeping all of it?** Token budget is finite. A 50-turn conversation would consume the entire context window, leaving no room for RAG results or tool outputs. Compression and selection ensure we keep what matters.

4. **How would you adapt this chatbot for a different website?** Change two things: (1) replace `VIZUARA_KNOWLEDGE_BASE` with content about your target site, and (2) update the system prompt. Everything else — the RAG pipeline, memory, tools, context assembler — works unchanged.

In [ ]:
#@title 🎧 Listen: Challenges
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_66_challenges.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Optional Challenges

1. **Semantic memory retrieval**: Instead of returning the most recent conversation history, embed each past turn and retrieve the most *semantically relevant* ones for the current query.

2. **Multi-hop tool chains**: Make the chatbot capable of calling multiple tools in sequence. For example: first call `faq_search` to check if the FAQ has an answer, and if not, call `course_lookup` as a fallback.

3. **Context window visualization**: Build a real-time widget that shows the context window contents as the conversation progresses, updating after each turn.

4. **Deploy to Gradio**: Wrap the `VizuaraChatbot` in a Gradio interface so anyone can interact with it through a web UI.

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_67_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final celebration
print()
print("╔" + "═" * 68 + "╗")
print("║                                                                    ║")
print("║   🎉  Congratulations!                                             ║")
print("║                                                                    ║")
print("║   You have built a complete context engineering pipeline:          ║")
print("║                                                                    ║")
print("║     ✅ Knowledge Base with web scraping + curated content          ║")
print("║     ✅ Vector Store with semantic embedding and retrieval          ║")
print("║     ✅ Conversation Memory with fact extraction                    ║")
print("║     ✅ Tool Registry with dynamic dispatch                        ║")
print("║     ✅ Context Assembler using ALL four strategies                 ║")
print("║     ✅ Working chatbot that feels genuinely intelligent            ║")
print("║                                                                    ║")
print("║   The model didn't change. Only the CONTEXT changed.              ║")
print("║   That is the power of context engineering.                        ║")
print("║                                                                    ║")
print("╚" + "═" * 68 + "╝")
print()
print("  Series Complete: Context Engineering for LLMs")
print("  Notebooks 1-5 by Vizuara")
print()
print("  Next step: swap vizuara.ai for YOUR website and")
print("  build a custom chatbot in under 10 minutes.")
print()

### Series Recap: Context Engineering for LLMs

Congratulations on completing all 5 notebooks! Here is what we covered:

| Notebook | Topic | Key Takeaway |
|----------|-------|-------------|
| **1** | Token Budget Management | The context window is finite — allocate tokens deliberately, not randomly |
| **2** | Context Failure Modes | Stale, conflicting, missing, and overflowing context silently degrade quality |
| **3** | RAG from Scratch | Embed, store, and retrieve — let the model see only what it needs |
| **4** | The Four Strategies | Write, select, compress, isolate — four tools to shape optimal context |
| **5** | Capstone Chatbot | All techniques compose into a system that feels like intelligence |

The core lesson: **LLM performance is bounded by context quality.** A perfectly tuned model with bad context will always lose to a decent model with excellent context. Context engineering is the highest-leverage skill in applied AI.

In [ ]:
# Final stats
print("FINAL PROJECT STATS")
print("=" * 60)
print(f"\n  Components built:        6")
print(f"  Knowledge base entries:  {len(VIZUARA_KNOWLEDGE_BASE)}")
print(f"  Chunks indexed:          {len(kb.chunks)}")
print(f"  Vector embeddings:       {vector_store.get_stats()['num_entries']}")
print(f"  Tools registered:        {len(tools.tools)}")
print(f"  Conversation turns:      {chatbot.memory.turn_count}")
print(f"  User facts extracted:    {len(chatbot.memory.key_facts)}")
print(f"  Profile fields:          {len(chatbot.memory.user_profile)}")
print()
print("  User profile: " + chatbot.memory.get_profile_summary())
print()
print("=" * 60)
print("  Thank you for learning with Vizuara!")
print("=" * 60)